# <font color='blue'>Data Science Academy - Machine Learning</font>

# <font color='blue'>Capítulo 10 - Processamento de Linguagem Natural</font>

****** Este Jupyter Notebook foi atualizado para a versão 3.6.1. da Linguagem Python em 05/07/2017 ******

## Sequence Labeling (Reconhecimento de Padrões)

Em Machine Learning, sequence labeling é um tipo de tarefa de reconhecimento de padrões que envolve a atribuição algorítmica de um rótulo categórico a cada membro de uma sequência de valores observados. Um exemplo comum de uma tarefa de rotulagem de sequência é parte do tagging de fala, que procura atribuir uma parte da fala a cada palavra em uma frase de entrada ou documento.

In [1]:
!pip install python-crfsuite

In [2]:
import pycrfsuite

In [3]:
from nltk.corpus import mac_morpho

In [4]:
list(mac_morpho.tagged_sents()[:10])

[[('Jersei', 'N'),
  ('atinge', 'V'),
  ('média', 'N'),
  ('de', 'PREP'),
  ('Cr$', 'CUR'),
  ('1,4', 'NUM'),
  ('milhão', 'N'),
  ('em', 'PREP|+'),
  ('a', 'ART'),
  ('venda', 'N'),
  ('de', 'PREP|+'),
  ('a', 'ART'),
  ('Pinhal', 'NPROP'),
  ('em', 'PREP'),
  ('São', 'NPROP'),
  ('Paulo', 'NPROP')],
 [('Programe', 'V'),
  ('sua', 'PROADJ'),
  ('viagem', 'N'),
  ('a', 'PREP|+'),
  ('a', 'ART'),
  ('Exposição', 'NPROP'),
  ('Nacional', 'NPROP'),
  ('do', 'NPROP'),
  ('Zebu', 'NPROP'),
  (',', ','),
  ('que', 'PRO-KS-REL'),
  ('começa', 'V'),
  ('dia', 'N'),
  ('25', 'N|AP')],
 [('Safra', 'N'),
  ('recorde', 'ADJ'),
  ('e', 'KC'),
  ('disponibilidade', 'N'),
  ('de', 'PREP'),
  ('crédito', 'N'),
  ('ativam', 'V'),
  ('vendas', 'N'),
  ('de', 'PREP'),
  ('máquinas', 'N'),
  ('agrícolas', 'ADJ')],
 [('A', 'ART'),
  ('degradação', 'N'),
  ('de', 'PREP|+'),
  ('as', 'ART'),
  ('terras', 'N'),
  ('por', 'PREP|+'),
  ('o', 'ART'),
  ('mau', 'ADJ'),
  ('uso', 'N'),
  ('de', 'PREP|+'),
  ('os',

In [5]:
# Obtendo 2.000 exemplos para facilitar a demonstração
tagged_sents = mac_morpho.tagged_sents()[:2000]

In [6]:
# Função para retronar as melhores features
def word2features(sent, i):
        word = sent[i]

        features = [
            'bias',
            'word.lower=' + word.lower(),
            'word[-3:]=' + word[-3:],
            'word[-2:]=' + word[-2:],
            'word.isupper=%s' % word.isupper(),
            'word.istitle=%s' % word.istitle(),
            'word.isdigit=%s' % word.isdigit(),
        ]
        if i > 0:
            word1 = sent[i - 1]
            features.extend([
                '-1:word.lower=' + word1.lower(),
                '-1:word.istitle=%s' % word1.istitle(),
                '-1:word.isupper=%s' % word1.isupper(),
            ])
        else:
            features.append('BOS')

        if i < len(sent) - 1:
            word1 = sent[i + 1]
            features.extend([
                '+1:word.lower=' + word1.lower(),
                '+1:word.istitle=%s' % word1.istitle(),
                '+1:word.isupper=%s' % word1.isupper(),
            ])
        else:
            features.append('EOS')

        return features
    
def sent2features(sent):
   return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
   return [label for token, label in sent]

def sent2tokens(sent):
   return [token for token, label in sent]

In [7]:
# Gerando dados de treino
sentences = list()
for sent in tagged_sents:
    sentences.append([token for token, tag in sent])

X_train = [sent2features(s) for s in sentences]
y_train = [sent2labels(s) for s in tagged_sents]

In [8]:
# Modelo
trainer = pycrfsuite.Trainer(verbose = False)
trainer.select(algorithm = 'lbfgs', type = 'crf1d')

In [9]:
for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)
trainer.train('pos.model')

In [10]:
tagger = pycrfsuite.Tagger()
tagger.open('pos.model')

In [11]:
sent = 'O menino jogou a bola azul no gol .'.split()
X_test = sent2features(sent)
guess = tagger.tag(X_test)
print (list(zip(sent, guess)))

[('O', 'ART'), ('menino', 'N'), ('jogou', 'V'), ('a', 'ART'), ('bola', 'N'), ('azul', 'ADJ'), ('no', 'N'), ('gol', 'ADJ'), ('.', 'N')]


### Fim

### Obrigado - Data Science Academy - <a href=http://facebook.com/dsacademy>facebook.com/dsacademybr</a>